# **Librerias**

In [11]:
import numpy as np
import pandas as pd

# **Parametros**

In [ ]:
# Clientes
clientes = [0, 1, 2, 3, 4, 5, 6]

# Matriz de distancias
dist = np.array(
    [
        #  0    1     2     3     4     5     6   ← hacia j
        [0, 32, 75, 64, 41, 89, 82],  # desde 0
        [32, 0, 49, 60, np.inf, 33, 58],  # desde 1
        [75, 49, 0, np.inf, 84, 50, 71],  # desde 2
        [64, 60, np.inf, 0, 58, 60, 68],  # desde 3
        [41, np.inf, 84, 58, 0, 66, 35],  # desde 4
        [89, 33, 50, 60, 66, 0, 39],  # desde 5
        [82, 58, 71, 68, 35, 39, 0],  # desde 6
    ],
    dtype=float,
)

# Demandas (unidades de carga). Entrega y recoleccion por comunidad

entrega = {0: 0, 1: 8, 2: 10, 3: 13, 4: 6, 5: 14, 6: 8}
recolec = {0: 0, 1: 6, 2: 4, 3: 7, 4: 26, 5: 5, 6: 15}

# Tiempos de servicio (min)
servicio = {0: 0, 1: 25, 2: 20, 3: 24, 4: 18, 5: 27, 6: 17}

# Capacidad y velocidad
capacidad = 40
vel_km_min = 0.8  # 0.8 km/min (48 km/h)

# **1) Matriz de ahorros**

In [ ]:
rutas = {i: [0, i, 0] for i in clientes if i != 0}
print(rutas)
print("")

# Matriz de ahorros
ahorros = np.zeros((len(clientes), len(clientes)))

lista_ahorros = []

for index_i, i in enumerate(clientes):
    for index_j, j in enumerate(clientes):
        if i != j:
            ahorros[index_i, index_j] = dist[i, 0] + dist[0, j] - dist[i, j]

print("Matriz de ahorros:")
print(pd.DataFrame(ahorros))

{1: [0, 1, 0], 2: [0, 2, 0], 3: [0, 3, 0], 4: [0, 4, 0], 5: [0, 5, 0], 6: [0, 6, 0]}

Matriz de ahorros:
     0     1      2     3     4      5      6
0  0.0   0.0    0.0   0.0   0.0    0.0    0.0
1  0.0   0.0   58.0  36.0  -inf   88.0   56.0
2  0.0  58.0    0.0  -inf  32.0  114.0   86.0
3  0.0  36.0   -inf   0.0  47.0   93.0   78.0
4  0.0  -inf   32.0  47.0   0.0   64.0   88.0
5  0.0  88.0  114.0  93.0  64.0    0.0  132.0
6  0.0  56.0   86.0  78.0  88.0  132.0    0.0


# **2) Lista de ahorros en orden descendente**

In [ ]:
# e. Lista de tuplas (i,j,ahorro) en orden descendente de ahorro
lista_ahorros = []
for i in range(len(clientes)):
    for j in range(len(clientes)):
        if (
            i != j
            and i != 0
            and j != 0
            and np.isfinite(ahorros[i, j])  # <-- solo ahorros válidos
            and ahorros[i, j] > float("-inf")  # <-- evita -inf
        ):
            if (i, j, int(ahorros[i, j])) not in lista_ahorros and (
                j,
                i,
                int(ahorros[j, i]),
            ) not in lista_ahorros:
                lista_ahorros.append((clientes[i], clientes[j], int(ahorros[i, j])))

lista_ahorros = sorted(lista_ahorros, key=lambda x: x[2], reverse=True)

for opcion in lista_ahorros:
    print(opcion)
print(len(lista_ahorros))

(5, 6, 132)
(2, 5, 114)
(3, 5, 93)
(1, 5, 88)
(4, 6, 88)
(2, 6, 86)
(3, 6, 78)
(4, 5, 64)
(1, 2, 58)
(1, 6, 56)
(3, 4, 47)
(1, 3, 36)
(2, 4, 32)
13


# **3) Heuristica**

## 3.1) Parametros y funciones auxiliares

In [ ]:
rutas_consolidadas = []  # Lista para almacenar las rutas consolidadas
which_route = {i: i for i in clientes if i != 0}  # Mapea cada cliente a su ruta inicial
rutas = {i: [0, i, 0] for i in clientes if i != 0}  # Rutas iniciales
contador = 0  # Contador de iteraciones
ahorro_total = 0  # Ahorro total acumulado

# * ========================================================
# * Funcion para verificar que no se viole la restriccion de demanda
# * ========================================================


def verificar_demanda(ruta, Q, entrega, recolec):
    # Salimos del deposito con la carga inicial correspondiente a las que voy a entregar
    # Verificar que estas entregas no excedan la capacidad del vehiculo
    carga = sum(entrega[cliente] for cliente in ruta if cliente != 0)

    if carga <= Q:
        # Ahora para cada cliente en la ruta, entrega y recolecta, verificar que despues de cada paso no se exceda la capacidad
        for n in ruta[1:-1]:  # Excluir el depósito (0) al inicio y al final
            carga -= entrega[n]  # Entrego lo que tengo que entregar
            carga += recolec[n]  # Recolecto lo que tengo que recolectar
            if carga > Q:  # Si en algun momento se excede la capacidad, retorno False
                return False

    return True  # Si nunca se excede la capacidad, retorno True

## 3.2) Funcion principal

In [ ]:
for i, j, ahorro in lista_ahorros:
    contador = contador + 1
    print("\n=========================================")
    print(f"Iteracion {contador}, par (i,j)=({i},{j}), ahorro={ahorro}")
    print("=========================================")

    # ? ========================================================
    # ? Verificacion 1: Los nodos i y j NO estan en la misma ruta
    # ? ========================================================

    if which_route[i] != which_route[j]:

        print(f"\nSe cumple verificacion 1: i={i} y j={j} no estan en la misma ruta")

        ruta_i = rutas[which_route[i]]  # Obtener la ruta actual de i
        ruta_j = rutas[which_route[j]]  # Obtener la ruta actual de j

        # ? =========================================================
        # ? Verificacion 2: Existen los enlaces (i,0) y (0,j)
        # ? =========================================================

        if ruta_i[-2] == i and ruta_j[1] == j:

            print(
                f"\nSe cumple verificacion 2: Existen los enlaces (i,0) y (0,j) para i={i} y j={j}"
            )

            ruta_original_i = which_route[
                i
            ]  # Guardar la ruta original de i antes de modificar
            ruta_original_j = which_route[
                j
            ]  # Guardar la ruta original de j antes de modificar

            nueva_ruta = ruta_i[:-1] + ruta_j[1:]  # Crear la nueva ruta consolidada,

            # ? =========================================================
            # ? Verificacion 3: No se viola ninguna restriccion
            # ? =========================================================

            if not verificar_demanda(nueva_ruta, capacidad, entrega, recolec):
                continue

            print(
                f"\nSe cumple verificacion 3: No se viola ninguna restriccion para la nueva ruta {nueva_ruta}"
            )

            # * =========================================================
            # * Verifiaciones exitosas, consolidar rutas
            # * =========================================================

            # Si todas las verificaciones son exitosas, consolidar las rutas agregando el arco (i,j) y eliminando (i,0) y (0,j)
            rutas[ruta_original_i] = (
                nueva_ruta  # Actualizar la ruta consolidada, ahora bajo la ruta de i
            )

            for cliente in nueva_ruta:
                if cliente != 0:
                    which_route[cliente] = which_route[
                        ruta_original_i
                    ]  # Actualizar el mapeo de rutas

            del rutas[ruta_original_j]  # Eliminar la ruta antigua de j

            rutas_consolidadas.append(nueva_ruta)

            ahorro_total += ahorro  # Actualizar el ahorro total
            print(
                f"\nConsolidando rutas: {ruta_i} y {ruta_j} -> Nueva ruta: {nueva_ruta}"
            )

print("\n-----------------------------------------")
print("\nRutas consolidadas durante el proceso:")
for ruta in rutas_consolidadas:
    print(ruta)
print("\nRutas finales:")
for ruta in rutas.values():
    print(ruta)


Iteracion 1, par (i,j)=(5,6), ahorro=132

Se cumple verificacion 1: i=5 y j=6 no estan en la misma ruta

Se cumple verificacion 2: Existen los enlaces (i,0) y (0,j) para i=5 y j=6

Se cumple verificacion 3: No se viola ninguna restriccion para la nueva ruta [0, 5, 6, 0]

Consolidando rutas: [0, 5, 0] y [0, 6, 0] -> Nueva ruta: [0, 5, 6, 0]

Iteracion 2, par (i,j)=(2,5), ahorro=114

Se cumple verificacion 1: i=2 y j=5 no estan en la misma ruta

Se cumple verificacion 2: Existen los enlaces (i,0) y (0,j) para i=2 y j=5

Se cumple verificacion 3: No se viola ninguna restriccion para la nueva ruta [0, 2, 5, 6, 0]

Consolidando rutas: [0, 2, 0] y [0, 5, 6, 0] -> Nueva ruta: [0, 2, 5, 6, 0]

Iteracion 3, par (i,j)=(3,5), ahorro=93

Se cumple verificacion 1: i=3 y j=5 no estan en la misma ruta

Iteracion 4, par (i,j)=(1,5), ahorro=88

Se cumple verificacion 1: i=1 y j=5 no estan en la misma ruta

Iteracion 5, par (i,j)=(4,6), ahorro=88

Se cumple verificacion 1: i=4 y j=6 no estan en la mism

## 3.3) Distancia total, capacidad maxima y tiempo de inicio y fin de cada ruta

In [ ]:

# * ========================================================
# * Calculo de la distancia total recorrida en las rutas finales
# * ========================================================


def distancia_total_rutas(rutas, dist):
    total_distancia = 0
    for ruta in rutas.values():
        distancia = sum(dist[ruta[k], ruta[k + 1]] for k in range(len(ruta) - 1))
        total_distancia += distancia
    return total_distancia


# * ========================================================
# * Calculo de la capacidad maxima utilizada en una ruta
# * ========================================================


def capacidad_maxima_utilizada(ruta, entrega, recolec):
    carga = sum(entrega[cliente] for cliente in ruta if cliente != 0)
    carga_maxima = carga  # Inicializar la carga máxima con la carga inicial

    for n in ruta[1:-1]:  # Excluir el depósito (0) al inicio y al final
        carga -= entrega[n]  # Entrego lo que tengo que entregar
        carga += recolec[n]  # Recolecto lo que tengo que recolectar
        if carga > carga_maxima:
            carga_maxima = carga  # Actualizar la carga máxima si es necesario

    return carga_maxima  # Retornar la carga máxima alcanzada durante la ruta


# * =======================================================
# * Calculo del tiempo de inicio y fin de cada ruta
# * =======================================================


def tiempo_inicio_fin(ruta, dist, servicio, vel_km_min):

    # Todas las rutas inician a las 7:00 AM
    tiempo_inicio = pd.Timestamp("07:00:00")  # Hora de inicio fija a las 7:00 AM
    tiempo_actual = tiempo_inicio

    # Para cada par de nodos en la ruta, calcular el tiempo de viaje y servicio
    for k in range(len(ruta) - 1):
        cliente_1, cliente_2 = ruta[k], ruta[k + 1]

        # Tiempo de viaje entre cliente_1 y cliente_2
        distancia = dist[cliente_1, cliente_2]
        tiempo_viaje = distancia / vel_km_min  # en minutos
        tiempo_actual += pd.Timedelta(minutes=tiempo_viaje)

        # Si destino no es el deposito, agregar tiempo de servicio
        if cliente_2 != 0:
            tiempo_actual += pd.Timedelta(minutes=servicio[cliente_2])

    tiempo_fin = tiempo_actual
    return tiempo_inicio, tiempo_fin


# * =======================================================
# * Resultados finales (Rutas, Distancia total, Ahorro total, Capacidad maxima utilizada, Tiempo inicio y fin)
# * =======================================================

distancia_total = distancia_total_rutas(rutas, dist)
print("\n-----------------------------------------")
print(f"\nDistancia total recorrida en las rutas finales: {distancia_total} km")
print(f"\nAhorro total acumulado durante el proceso: {ahorro_total} km")
print("\n-----------------------------------------")


print("\nCapacidad máxima utilizada en cada ruta final:")
for ruta in rutas.values():
    carga_max = capacidad_maxima_utilizada(ruta, entrega, recolec)
    print(f"Ruta: {ruta} | Capacidad máxima utilizada: {carga_max}")
    
    
print("\n-----------------------------------------")
print("\nTiempo de inicio y fin de cada ruta final:")
for ruta in rutas.values():
    tiempo_inicio, tiempo_fin = tiempo_inicio_fin(ruta, dist, servicio, vel_km_min)
    print(
        f"Ruta: {ruta} | Tiempo inicio: {tiempo_inicio.time()} | Tiempo fin: {tiempo_fin.time()}"
    )


-----------------------------------------

Distancia total recorrida en las rutas finales: 415.0 km

Ahorro total acumulado durante el proceso: 351 km

-----------------------------------------

Capacidad máxima utilizada en cada ruta final:
Ruta: [0, 1, 2, 5, 6, 0] | Capacidad máxima utilizada: 40
Ruta: [0, 3, 4, 0] | Capacidad máxima utilizada: 33

-----------------------------------------

Tiempo de inicio y fin de cada ruta final:
Ruta: [0, 1, 2, 5, 6, 0] | Tiempo inicio: 07:00:00 | Tiempo fin: 13:44:00
Ruta: [0, 3, 4, 0] | Tiempo inicio: 07:00:00 | Tiempo fin: 11:05:45
